In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
sns.set()

In [2]:
dataraw = pd.read_csv("healthcare-dataset-stroke-data.csv")
dataraw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [3]:
dataraw.dropna(axis = 0, inplace=True)

In [4]:
dataraw.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [5]:
dataraw.drop(columns= ['id'], axis = 1, inplace= True)
dataraw.describe(include = 'all')

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
count,4909,4909.000000,4909.000000,4909.000000,4909,4909,4909,4909.000000,4909.000000,4909,4909.000000
unique,3,NaN,NaN,NaN,2,5,2,NaN,NaN,4,NaN
top,Female,NaN,NaN,NaN,Yes,Private,Urban,NaN,NaN,never smoked,NaN
freq,2897,NaN,NaN,NaN,3204,2811,2490,NaN,NaN,1852,NaN
mean,NaN,42.865374,0.091872,0.049501,NaN,NaN,NaN,105.305150,28.893237,NaN,0.042575
std,NaN,22.555115,0.288875,0.216934,NaN,NaN,NaN,44.424341,7.854067,NaN,0.201917
min,NaN,0.080000,0.000000,0.000000,NaN,NaN,NaN,55.120000,10.300000,NaN,0.000000
25%,NaN,25.000000,0.000000,0.000000,NaN,NaN,NaN,77.070000,23.500000,NaN,0.000000
50%,NaN,44.000000,0.000000,0.000000,NaN,NaN,NaN,91.680000,28.100000,NaN,0.000000
75%,NaN,60.000000,0.000000,0.000000,NaN,NaN,NaN,113.570000,33.100000,NaN,0.000000


In [6]:
dataraw.reset_index(drop= True, inplace = True)

In [7]:
data = pd.get_dummies(dataraw, drop_first= True)
data.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,1,0,1,0,1,0,0,1,1,0,0
1,80.0,0,1,105.92,32.5,1,1,0,1,0,1,0,0,0,0,1,0
2,49.0,0,0,171.23,34.4,1,0,0,1,0,1,0,0,1,0,0,1
3,79.0,1,0,174.12,24.0,1,0,0,1,0,0,1,0,0,0,1,0
4,81.0,0,0,186.21,29.0,1,1,0,1,0,1,0,0,1,1,0,0


In [8]:
y = data['stroke']
x = data.drop(axis = 1, columns = ['stroke'])
x.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,1,0,0,1,1,0,0
1,80.0,0,1,105.92,32.5,1,0,1,0,1,0,0,0,0,1,0
2,49.0,0,0,171.23,34.4,0,0,1,0,1,0,0,1,0,0,1
3,79.0,1,0,174.12,24.0,0,0,1,0,0,1,0,0,0,1,0
4,81.0,0,0,186.21,29.0,1,0,1,0,1,0,0,1,1,0,0


In [9]:
x1 = sm.add_constant(x)
reg_log = sm.Logit(y, x1)
results_log = reg_log.fit()
results_log.summary()

         Current function value: 0.138850
         Iterations: 35


d:\PYTHON\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 stroke   No. Observations:                 4909
Model:                          Logit   Df Residuals:                     4892
Method:                           MLE   Df Model:                           16
Date:                Mon, 13 Mar 2023   Pseudo R-squ.:                  0.2113
Time:                        19:51:46   Log-Likelihood:                -681.61
converged:                      False   LL-Null:                       -864.19
Covariance Type:            nonrobust   LLR p-value:                 7.107e-68
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -8.3166      0.666    -12.495      0.000      -9.621      -7.012
age                                0.0735      0.006     11.578      0.000       0.061       0.086
hypertension                       0.5249      0.175      2.999      0.003       0.182       0.868
heart_disease                      0.3488      0.207      1.683      0.092      -0.057       0.755
avg_glucose_level                  0.0047      0.001      3.595      0.000       0.002       0.007
bmi                                0.0041      0.012      0.342      0.732      -0.019       0.027
gender_Male                       -0.0146      0.154     -0.095      0.925      -0.317       0.288
gender_Other                     -15.2670    2.8e+04     -0.001      1.000   -5.49e+04    5.48e+04
ever_married_Yes                  -0.1152      0.247     -0.466      0.641      -0.600       0.370
work_type_Never_worked            -9.0987    498.163     -0.018      0.985    -985.480     967.282
work_type_Private                  0.1608      0.223      0.720      0.472      -0.277       0.599
work_type_Self-employed           -0.2642      0.254     -1.038      0.299      -0.763       0.234
work_type_children                 0.6817      1.114      0.612      0.541      -1.502       2.865
Residence_type_Urban               0.0045      0.150      0.030      0.976      -0.289       0.298
smoking_status_formerly smoked     0.2753      0.247      1.114      0.265      -0.209       0.760
smoking_status_never smoked        0.2081      0.232      0.895      0.371      -0.247       0.664
smoking_status_smokes              0.5893      0.266      2.211      0.027       0.067       1.112
==================================================================================================
"""

In [31]:
x1.head()

,const,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1.0,67.0,0,1,228.69,36.6,1,0,1,0,1,0,0,1,1,0,0
1,1.0,80.0,0,1,105.92,32.5,1,0,1,0,1,0,0,0,0,1,0
2,1.0,49.0,0,0,171.23,34.4,0,0,1,0,1,0,0,1,0,0,1
3,1.0,79.0,1,0,174.12,24.0,0,0,1,0,0,1,0,0,0,1,0
4,1.0,81.0,0,0,186.21,29.0,1,0,1,0,1,0,0,1,1,0,0


In [43]:
def confusion_matrix(data,actual_values,model):
        pred_values = model.predict(data)
        bins=np.array([0,0.5,1])
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        return cm, accuracy

In [44]:
confusion_matrix(x1,y,results_log)

(array([[4.699e+03, 1.000e+00],
        [2.080e+02, 1.000e+00]]),
 0.9574251375025463)

In [55]:
gender = input("Enter your gender: ")
while (gender!= "Male" and gender!= "Female" and gender != "Other"):
    gender = input("Enter your gender again: ")
    
age = float(input("Enter your age: "))
while (age<=0):
    age = float(input("Enter your age again: "))


hypertension = int(input("Does the patient have hypertension? Enter 1 for Yes and 0 for NO: "))
while (hypertension!= 1 and hypertension!= 0):
    hypertension = int(input("Does the patient have hypertension? Enter 1 for Yes and 0 for NO again: "))


heartdisease = int(input("Does the patient have heartdisease? Enter 1 for Yes and 0 for NO: "))
while (heartdisease!= 1 and heartdisease!= 0):
    hypertension = int(input("Does the patient have heartdisease? Enter 1 for Yes and 0 for NO again: "))


marry_status = input("Is the patient married? Enter Yes or No: ")
while (marry_status!= "Yes" and marry_status!= "No"):
    marry_status = input("Is the patient married? Enter Yes or No again: ")


work_type = input("Where does the patient work? Private, self-employed or other: ")


resid = input("Where does the patient live? Rural or residential area: ")
while (resid!= "Rural" and resid!= "Urban"):
    resid = input("Where does the patient live? Rural or residential area. Try again: ")


glucose = float(input("What is patient's average glucose level: "))
while (glucose<= 20 or glucose >= 500):
    glucose = float(input("What is patient's average glucose level. Try again: "))


bmi = float(input("What is patient's average bmi level: "))
while (bmi<=0):
    bmi = float(input("What is patient's average bmi level. Try again: "))


smoking_status = input("Does the patient smoke? Enter Unknown, Smokes. Formerly Smoked or never smoked: ")
while (smoking_status!= "unknown" and smoking_status!= "smokes" and smoking_status != "never smoked" and smoking_status != "formerly smoked"):
    smoking_status = input("Does the patient smoke? Enter Unknown, Smokes. Formerly Smoked or never smoked. Try again: ")


temp = {'const': [1.0], 'gender':[gender], 'age': [age], 'hypertension': [hypertension], 'heart_disease': [heartdisease],'ever_married': [marry_status], 'work_type': [work_type], 'Residence_type': [resid], 'avg_glucose_level': [glucose], 'bmi': [bmi], 'smoking_status': [smoking_status]}
x_raw = dataraw.drop(axis = 1, columns = ['stroke'])
x_raw['const'] = 1.0
first_column = x_raw.pop('const')
x_raw.insert(0, 'const', first_column)
testdf = pd.DataFrame(temp)
testdffinal_tp = x_raw.append(testdf)
testdffinal = pd.get_dummies(testdffinal_tp, drop_first=True)

predi = results_log.predict(testdffinal.iloc[-1].values.reshape(1,-1))

print("Predicted Value is: ", predi)

if predi[0]>0.5:
    print("Stroke")
else:
    print("No Stroke")

Predicted Value is:  [0.7143049]
Stroke


C:\Users\Haris Rehman\AppData\Local\Temp\ipykernel_263064\475231129.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  testdffinal_tp = x_raw.append(testdf)
